In [ ]:
!pip install PuLP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import copy
import pandas as pd

from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
from pandas.core.internals.array_manager import PandasArray

### Исходные данные задачи

1. Максимизируем прибыль <br> $z_{1} = 8x_{1} + 7x_{2} \to max$ <br> <br>
2. Минимизируем издержки простоя оборудования <br> $z_{2} = -34x_{1} - 24x_{2} + 1248 \to min $ <br> <br>
3. Ограничения:
<br>
$f(n) =
  \begin{cases}
    4x_{1} + 3x_{2} \leq 144   \\
    2x_{1} + x_{2} \leq 64 \\
    2x_{1} + 3x_{2} \leq 120 \\
    x_{1}\geq0, x_{2}\geq0
  \end{cases}
$
<br>


In [ ]:
criterias = pd.DataFrame(np.array([[8, 7], [-34, -24]]), columns=range(2))
criterias['direct'] = ['max', 'min']
constraints = pd.DataFrame(np.array([[4, 3], [2, 1], [2, 3]]), columns=range(2))
bs = [144, 64, 120]
add_b = [0,1248]

### Метод последовательный уступок

In [ ]:
def mpu(c_df, a_df, bs, rebate_coef, low, up):
    var_xs = LpVariable.dicts('x', c_df.columns.values[:-1], lowBound=low, upBound=up)
    is_not_First = False
    cur_bF = -1
    add_consts_list = []
    for i in c_df.index.values: # проходим по всем функциям
        # Решаем злп с помощью встроенных функций
        direct = LpMaximize if c_df.loc[i, 'direct'] == 'max' else LpMinimize
        model = LpProblem(name=f'Main_criteria', sense=direct)
        obj = lpSum([c_df.loc[i, x] * var_xs[x] for x in var_xs])
        model += lpSum(obj)
        if is_not_First:
            new_const = [c_df.loc[i - 1, x] * var_xs[x] for x in var_xs]
            print(f'Размер уступки = {rebate_coef}')
            rebate =  rebate_coef
            # Добавляем дополнительное ограничение
            if direct == LpMaximize:
                add_const = lpSum(new_const) >= cur_bF - rebate
                print(add_const)
            else:
                add_const = lpSum(new_const) <= cur_bF + rebate
            add_consts_list.append(add_const)
            for c in add_consts_list:
                model += c

        for j in a_df.index.values:
            cur_const = [a_df.loc[j, x] * var_xs[x] for x in var_xs]
            model += lpSum(cur_const) <= bs[j]
        model.solve()
        cur_bF = model.objective.value()
        bs.append(cur_bF)
        is_not_First = True
        print(f'Значение целевой функции = {cur_bF}')
        for var in model.variables():
            print(f'{var.name}: {var.value()}')
        print()

    return model.variables(), cur_bF

In [ ]:
vs,f = mpu(criterias, constraints, bs, rebate_coef=8, low=0, up=100)

Значение целевой функции = 320.0
x_0: 12.0
x_1: 32.0

Размер уступки = 8
Значение целевой функции = -1200.0
x_0: 24.0
x_1: 16.0



In [ ]:
fs = []
xs = []
for var in vs: xs.append(var.varValue)
for t in criterias.drop('direct', axis=1).values: print('function with coefs',t,' result ', np.dot(t, xs))

function with coefs [8 7]  result  304.0
function with coefs [-34 -24]  result  -1200.0


### Метод главного критерия

In [ ]:
def mgk(c_df, a_df, bs, b_not_main, low=0, up=100, idx_main=0):
    f_idx_to_constraint = list(set(range(c_df.shape[0])) - set([idx_main]))
    print(f_idx_to_constraint)
    var_xs = LpVariable.dicts('x', c_df.columns.values[:-1], lowBound=low, upBound=up)
    direct = LpMaximize if c_df.loc[idx_main, 'direct'] == 'max' else LpMinimize
    model = LpProblem(name=f'Main_criteria', sense=direct)
    obj = lpSum([c_df.loc[idx_main, x] * var_xs[x] for x in var_xs])
    model += lpSum(obj)

    for i in f_idx_to_constraint:
        new_const = [c_df.loc[i, x] * var_xs[x] for x in var_xs]

        if c_df.loc[i, 'direct'] == 'max':
            model += lpSum(new_const) >= b_not_main[i]
        else:
            model += lpSum(new_const) <= b_not_main[i]

    for j in a_df.index.values:
        cur_const = [a_df.loc[j, x] * var_xs[x] for x in var_xs]
        model += lpSum(cur_const) <= bs[j]

    model.solve()
    cur_bF = model.objective.value()

    print(f'cur_solve = {cur_bF}')
    for var in model.variables():
        print(f'{var.name}: {var.value()}')

    return model.variables(), cur_bF

In [ ]:
vs,f = mgk(criterias, constraints, bs, b_not_main={1:1248}, low=10, up=50, idx_main=0)

[1]
cur_solve = 320.0
x_0: 12.0
x_1: 32.0


In [ ]:
fs = []
xs = []

for var in vs: xs.append(var.varValue)
for t in criterias.drop('direct', axis=1).values: print('function with coefs', t,' result ', np.dot(t, xs))

function with coefs [8 7]  result  320.0
function with coefs [-34 -24]  result  -1176.0
